In [ ]:
from skimage import io
import pickle
import sys
import numpy as np
import copy
sys.path.append('/Users/jerrytang/.virtualenvs/cv/lib/python2.7/site-packages')

import helpers
import cluster
import evaluation
import registration
import processing
import correction
import splicing

## Initialization

#### Design Parameters

In [ ]:
# splicing
segment_length = 50
overlap_length = 10

# clustering
cluster_cutoff_distance = 0.7
cluster_accept_size = 0.0005

# correction
max_interpolation_length = 20

#### Input Paths

In [ ]:
files = ['/Users/jerrytang/hydra_data/corrected_100.csv']
video_path = '/Users/jerrytang/hydra_data/stk_0001_Substack (1-5000).tif'
assignments_read_path = '/Users/jerrytang/hydra_data/motion_assignments.csv'
# None if the loaded data set is not motion corrected (corrected map is output of ICY motion correction)
motion_correction_map_path = '/Users/jerrytang/hydra_data/map_100.csv'

#### Output Paths

In [ ]:
# res is 2 for 512 images, 1 for 1024
res = 2 
video_save_folder = '/Users/jerrytang/hydra_data/temp_video'
assignments_save_path = '/Users/jerrytang/hydra_data/corrected_assignments.csv'

## Tracking

In [ ]:
import pandas as pd
import numpy as np
full = helpers.load_files(files)[0:100]

In [ ]:
vid = io.imread(video_path)[0:100]

In [ ]:
if motion_correction_map_path != None:
        correction_map = processing.load_correction_map("/Users/jerrytang/0_hydra_data/map_100.csv")
corrected_full = processing.reverse_full_correction(full, correction_map)
for i in range(len(corrected_full)):
    corrected_full[i] = np.asarray(corrected_full[i])

In [ ]:
all_tracks = []

segment_length = 50
overlap_length = 10
    
total_length = 0
vid_end = len(full)
while(total_length < vid_end):
    if total_length == 0:
        start = 0
        end = segment_length
    else: 
        start = total_length - overlap_length
        end = min(vid_end, total_length + segment_length - overlap_length)
    new_tracks = splicing.get_tracks_soft(start, end, overlap_length, full, cluster_cutoff_distance, cluster_accept_size, max_interpolation_length, correction_map, corrected_full)
    if total_length == 0:
        all_tracks = new_tracks
        total_length += segment_length
    else: 
        splicing.splice(all_tracks, new_tracks, total_length, segment_length, overlap_length)
        total_length += segment_length - overlap_length

In [ ]:
# get distribution of path lengths
import matplotlib.pyplot as plt
% matplotlib inline
plt.hist(list(map(lambda x : len(x), all_tracks)))

In [ ]:
assignments_save_path = '/Users/jerrytang/hydra_data/final_hard_stitched.csv'
helpers.write_neuron_assignments(all_tracks, assignments_save_path, 1, False)

In [ ]:
assigned = np.zeros(100)
for neuron in all_tracks:
    for i in neuron.keys():
        assigned[i] += 1
        
import matplotlib.pyplot as plt
% matplotlib inline
plt.scatter(range(100), assigned)